In [ ]:
import os
import pandas as pd
import numpy as np
import sys
import win32com.client
import getpass
import datetime
import pywintypes
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

# Function

In [ ]:
pd.set_option('display.width', None)
pd.set_option('display.max_column',None)
pd.set_option('display.max_rows',None)

def drop_by_index(X,indexes):
    """
    helper function to drop rows of dataframe and return new dataframe without those rows with indexes resetted
    """
    X = X.drop(indexes)
    X = X.reset_index().drop(columns="index")
    return(X)

def getDataToDF(xlws,start,end,first = False):
    '''
    input: excel worksheet path
    start: rows to begin
    end: rows to stop
    first: used for when doing getting data using bacthing
            true when the record is first wave
            false when the record is after first wave
    
    1. years that are more than the current year are replaced into 19xx. eg 2020 => 1920
    2. drop all dx date = 2020-01-01 (input errors)
    3. drop all rows where dx_Date is empty
    
    '''
    
    last_col = xlws.UsedRange.Columns.Count
    header = xlws.Range(xlws.Cells(1, 1), xlws.Cells(1, last_col)).Value
    content = xlws.Range(xlws.Cells(start, 1), xlws.Cells(end, last_col)).Value
    
    if first:
        data = list(content[1:])
    else:
        data = list(content[0:])
        
    for x in range(0,len(data)):
        data[x] = list(data[x])
        for y in range(0,len(data[x])):
            if isinstance(data[x][y], pywintypes.TimeType):
                temp = str(data[x][y]).rstrip("+00:00").strip()
                if temp[:2] == '20' and int(temp[:4]) >= int(now.year):
                    temp = '19' + temp[2:]
                data[x][y] = datetime.datetime.strptime(temp, "%Y-%m-%d")
    df = pd.DataFrame(data, columns=header[0])
    
    df.rename(columns={"dob_new":"dob","Age_new":"Age_@_Dx"}, inplace = True)
    
    df.fillna(value=pd.np.nan, inplace=True)
    
    #drop all rows where dx_date is empty
    df.drop(df[df["dx_date"].isnull()].index, inplace=True)

    return df

def processCol(df):                
            
    #for those dropdown variables in dictionary
    toDoDict = {}
    toDoDict['Gender'] = [('1','2','4','5','6','9997','9998','9999'),
                          ['Male','Female','Transsexual, NOS','Transsexual, natal male','Transsexual, natal female','NA',
                           'Others (intersex, disorders of sexual development/DSD)','Unknown']]
    toDoDict['c_tstage'] = [('1','2','3','4','5','6','7','8','9','11','12','13','41','42','43','44'),
                            ['T1','T2','T3','T4','Tx','T0','Tis','T1mic','unknown','T1a','T1b','T1c',\
                             'T4a','T4b','T4c','T4d']]
    toDoDict['cNstage'] = [('1','2','3','4','9','10','31','32','41','42','43'),
                           ['N0','N1','N2','N3','Nx','unknown','N2a','N2b','N3a','N3b','N3c']]
    toDoDict['cMstage'] = [('0','1','2','3','4','5','6','9997','9999'),
                           ['MX','M0','M1','M1a','M1b','M1c','M2','NA','Unknown']]     
    toDoDict['c_Staging'] = [('0','1','2','3','4','5','6','7','9','10','11','12','31','41'),
                             ['DCIS/LCIS non-invasive','Stage 1','Stage 2A','Stage 2B',\
                              'Stage 3A','Stage 3B','Stage 3C','Stage 4','Unknown','Stage 0',\
                              'Stage 1A','Stage 1B','Stage 2','Stage 3']] 
    toDoDict['tstage'] = [('1','2','3','4','5','6','7','8','9','11','12','13','41','42','43','44'),
                          ['T1','T2','T3','T4','Tx','T0','Tis','T1mic','unknown','T1a','T1b','T1c',\
                           'T4a','T4b','T4c','T4d']] 
    toDoDict['nstage'] = [('1','2','3','4','5','6','9','10','21','22','23','31','32','41','42','43'),
                          ['N0','N1','N2','N3','N1mic','N0 (i+)','Nx','unknown','N1a','N1b','N1c',\
                           'N2a','N2b','N3a','N3b','N3c']]
    toDoDict['Mstage'] = [('1','2','3','9'),('M0','M1','Mx','unknown')] 
    toDoDict['p_Staging'] = [('0','1','2','3','4','5','6','7','9','10','11','12','31','41'),
                             ['DCIS/LCIS non-invasive','Stage 1','Stage 2A','Stage 2B',\
                              'Stage 3A','Stage 3B','Stage 3C','Stage 4','Unknown','Stage 0',\
                              'Stage 1A','Stage 1B','Stage 2','Stage 3']]
    toDoDict['diff'] = [('0','1','2','3','9'),['grade 0','grade 1','grade 2','grade 3','unknown']] 
    toDoDict['TNM_Stage'] = [('0','1','2','3','4','5','6','7','9','10','11','12','13','31'),
                             ['DCIS/LCIS non-invasive','stage 1','stage 2A','stage 2B',\
                              'stage 3A','stage 3B','stage 3C','stage 4','Unknown','Stage 0',\
                              'stage 1A','stage 1B','Stage 3','stage 2']]
    toDoDict['ProgStage_AJCC8'] = [('0','4','11','12','21','22','31','32','33'),
                                   ['Stage 0','Stage IV','Stage IA','Stage IB','Stage IIA',\
                                   'Stage IIB','Stage IIIA','Stage IIIB','Stage IIIC']] 
    toDoDict['ER'] = [('1','2','3','4'),['positive','negative','unknown','Equivocal']] 
    toDoDict['PR'] = [('1','2','3','4'),['positive','negative','unknown','Equivocal']]
    toDoDict['cerbB2'] = [('1','2','3','4'),['positive','negative','unknown','Equivocal']]
    toDoDict['Her2'] = [('1','2','3','4','9'),['Positive','Negative','Not Done','Equivocal','Unknown']]
    toDoDict['cause_of_death'] = [('1','2','9'),['breast cancer related','N','unknown']]
    toDoDict['Count_as_DFS'] = [('0','1'),['N' ,'RECURRENCE (any)']]
    toDoDict['Count_as_OS'] = [('0','1'),['N' ,'Dead']]
    
    for q in list(['T (no subgroup)', 'N (no subgroup)', 'M (no subgroup)','T','N','M','Stage']):
        df[q] = df[q].str.lower()
    
    for k,v in toDoDict.items():
        '''
        1. convert numeric codes into category to standardise
            k: column name
            v[0]: numeric representation
            v[1]: category
        2. drop all those values that are outside of the defined numeric codes 
        '''
        
        v[1] = [x.lower() for x in v[1]]
        df[k].replace(to_replace =v[0],\
                      value = v[1],\
                      inplace = True)
        df[k] = df[k].str.lower()
        
        # allow null values
        v[1].append(pd.np.nan)
        
        # drop abnormal values   
        df.drop(df.loc[~df[k].isin(v[1])].index, inplace=True)
    
    DateList =  {"death_age":('death','dob')}
    for k,v in DateList.items():
        df[v[0]] = pd.to_datetime(df[v[0]])
        df[v[1]] = pd.to_datetime(df[v[1]])
        df[k] = (df[v[0]] - df[v[1]]).dt.days
        df[k] = df[k].floordiv(365.2425 , fill_value = pd.np.nan) 
        
        df = df.drop(columns=v[0])
        
        #drop negative age (input error)
        df.drop(df[df[k] < 0].index, inplace=True)
        
    #drop all rows where death age is lesser than age @ dx
    df.drop(df[df['death_age'] < df['Age_@_Dx']].index, inplace=True)
    
    return df

def dropColCDM(df, listToDrop):
    df = df.drop(columns=listToDrop)
    return df


# Main Process

In [ ]:
FileToCheck = 'C:\\SMU_v2\\clinical_output.pkl'
Clinical_Path = "C:\\SMU_v2\\Clinical Data_Masked_v3_updated050220.xlsx"
if os.path.exists(FileToCheck):
    CDM = pd.read_pickle(FileToCheck)
else:
    # primary set up
    xlApp = win32com.client.Dispatch("Excel.Application")
    xlApp.Interactive = False
    xlApp.Visible = False

    #require user input for password
    pwd = getpass.getpass('Enter file password: ')

    now = datetime.datetime.now()
    
    xlwb = xlApp.Workbooks.Open(Clinical_Path, False, True, None, pwd)
    xlws = xlwb.Worksheets(1) 
    last_row = xlws.UsedRange.Rows.Count

    CDM = getDataToDF(xlws,1,last_row)

    CDM = processCol(CDM)
    
    CDM['nodespos'].replace({'NA': pd.np.NaN}, inplace=True)
    CDM['size_precise'].replace({'unknown': pd.np.NaN}, inplace=True)
    
    ##type casting to save space
    col_list = list(["Gender","c_tstage","cNstage", "cMstage","c_Staging","tstage","nstage","Mstage",\
                     "p_Staging","diff","TNM_Stage","ProgStage_AJCC8","ER","PR","cerbB2",\
                     "Her2","cause_of_death","Count_as_DFS","Count_as_OS","Count_as_CSS",\
                     'T (no subgroup)', 'N (no subgroup)', 'M (no subgroup)',\
                     'T','N','M','Stage', 'Size'])
    for i in col_list:
        CDM.loc[:,i] = CDM[i].astype("category")

    CDM.loc[:,"size_precise"] = CDM["size_precise"].astype("float32")
    CDM.loc[:,"nodespos"] = CDM["nodespos"].astype("float16")
    
    CDM.to_pickle(FileToCheck)
    
    #reset variables
    xlws = None
    xlwb.Close(False)
    xlwb = None

    #remove buffer and reset system settings
    xlApp.Interactive = True
    xlApp.Visible = True
    xlApp.Quit()
    xlApp = None

In [ ]:
def dataSetting(dropCol,FILE_FOLDER = "C:\\SMU_v2\\"):
    '''
    function to read the pkl from from datasource
        1. Remove dx_date that is NULL.
        2. Drop all rows where crucial fields for X_features are NULL.
        3. Convert Date columns into datetime format
        4. Derive OS, CSS, DFS days based on dx_date
        5. Create status column to indicate if the patient is dead or alive base on if death_age exists
    '''
    df = pd.read_pickle(FILE_FOLDER + "clinical_output.pkl").reset_index().drop(columns="index")
    to_drop = df[df['dx_date']=="NA"].index
    df = drop_by_index(df,to_drop)

    df.drop(columns=dropCol,inplace = True)

    # drop all rows where dates are null
    df.dropna(axis=0,\
                    subset=['Date_for_DFS','Date_for_OS','Date_for_CSS','dx_date','Age_@_Dx'],\
                    inplace=True)
    
    # convert all datetime in dataframe into dateime format for processing
    df["Date_for_DFS"] = pd.to_datetime(df["Date_for_DFS"])
    df["Date_for_OS"] = pd.to_datetime(df["Date_for_OS"])
    df["Date_for_CSS"] = pd.to_datetime(df["Date_for_CSS"])
    df["dx_date"] = pd.to_datetime(df["dx_date"])
    df['last_seen']= pd.to_datetime(df["dx_date"])
    df['dob']= pd.to_datetime(df["dx_date"])

    # calculate in days
    df["DFS_days"] = (df["Date_for_DFS"] - df['dx_date'] )/np.timedelta64(1, 'D')
    df["OS_days"] = (df["Date_for_OS"] - df['dx_date'] )/np.timedelta64(1, 'D')
    df["CSS_days"] = (df["Date_for_CSS"] - df['dx_date'] )/np.timedelta64(1, 'D')

    # alive or dead
    df['status'] = np.where(df['Count_as_OS'] == "dead", False, True)
    
#     # age
#     df.loc[df['Age_@_Dx'].isnull() ,'Age_@_Dx'] = round((df['dx_date'] - df['dob']).dt.days/365.25,1)
    
    return df
def ComputeYears(df, Year_list):
    '''
    Create a list to contain df for different years of survival
    The df will filter those patient that has deceased or days of survival longer than the defined years.
    '''

    df_dict = {}

    for i in Year_list:
        tmp = {}
        for x in list(["DFS", "CSS", "OS"]):
            df['{}_{}_years'.format(x, i)] = np.where(
                                                      np.logical_or(df['death_age'] > 0,\
                                                      df['{}_days'.format(x)]/(365.25*i) >= i),\
                                                      True,False)
            tmp[x] = df[df['{}_{}_years'.format(x, i)] == True]
        df_dict['{}_years'.format(i)] = tmp
    return df_dict
def settingXY(df, X_features, Y_features, OHE_LOCATION = "C:\\SMU_v2\\OHE\\", name=""):
    '''
    This function returns the X and Y features need for model training
        - The function also generates one pkl that contains the One Hot Encoder for new raw data 
    
    X_features = features to use for X
    Y_features = features to use for Y 
    YEAR = years of patient record interested
    SYTPE = survival type (OS, DFS, CSS)
    OHE_LOCATION = location to store the pkl file
    '''
    import pickle
    from sksurv.preprocessing import OneHotEncoder

    X = df[X_features]
    Y = df[Y_features]

    # Save enconder so that we can OHE new data
    enc = OneHotEncoder()
    enc.fit(X)
    
    # OHE for probability
    X = enc.transform(X)
    with open(OHE_LOCATION + name + '_encoder.pickle', 'wb') as f:
        pickle.dump(enc, f) 
                  
    # convert Y to structured array
    s = Y.dtypes
    Y = np.array([tuple(x) for x in Y.values], dtype=list(zip(s.index, s)))
   
    return X, Y
def layeredData(df, group_dict,y_features, YEAR, STYPE):
    
    '''
        this function generates the dataframe required for specific groups we hope to analyze
        there are total 3 different groups but group 3 consist of multiple subgroups which leads a total of 5
        dataframe.
        Group 1: patient with stage 4 cancer
        Group 2: patient which unknown records or at initial diagnosis stage
        Group 3: make up of patient that does not belong to the groups above
    '''
    model_data_dict = {}
    TO_USE = df['{}_years'.format(YEAR)][STYPE]
    
    print("Overall initial size: {} \n".format(TO_USE.shape[0]))
        
    for key,value in group_dict.items():
        TO_USE_COPY = TO_USE.copy()

        tmp = {}
        
        waves = value['wave']
    
        if key != "group 3":
            # for group 1 and group 2 select rows that contains either stage 4/non invasive in Stage
            TO_USE_COPY = TO_USE_COPY.loc[TO_USE_COPY['Stage'] == group_dict[key]['stage'][0]]
        else:
            # for group 3 do not select rows that contains either stage 4 or non invasive in c_Staging or p_Staging
            stage = np.logical_and(TO_USE_COPY['Stage'] != group_dict[key]['stage'][0],\
                                    TO_USE_COPY['Stage'] != group_dict[key]['stage'][1])
            
            TO_USE_COPY = TO_USE_COPY.loc[stage]
            
        print("{} data size: {}".format(key,len(TO_USE_COPY)))
        
        for wave in waves:
            TO_USE_COPY2 = TO_USE_COPY.copy()
            TO_USE_COPY2 = TO_USE_COPY2[waves[wave] + y_features]
            
            len_before = len(TO_USE_COPY2)
            print("\t{} data size before dropping nan: {}".format(wave,len_before))
            
            TO_USE_COPY2.dropna(axis=0,subset=waves[wave]+ y_features, inplace=True)
            TO_USE_COPY2.reset_index(drop=True)

            len_after = len(TO_USE_COPY2)
            print("\t\t after dropping nan: {}".format(len_after))
            
            for i in waves[wave]:
                if not (i in ['nodespos','Age_@_Dx','size_precise']):
                    TO_USE_COPY2.loc[:,i] = TO_USE_COPY2[i].astype("category")
                else:
                    TO_USE_COPY2.loc[:,i] = TO_USE_COPY2[i].astype("float32")
            
            X, Y = settingXY(TO_USE_COPY2, waves[wave], y_features,name= "{}_{}".format(key,wave))   

            tmp[wave] = {
                            "X": X,\
                            "Y":Y      
                        }    
    
        model_data_dict[key] = tmp
    return model_data_dict

listToDrop = ['NRIC','dob','Has Bills?','Side','Hospital','KKH','NCCS','SGH','END_OF_ENTRY']
CDM = dataSetting(listToDrop)
year_list = list([1,5,10])

# only return data that has longer timeframe than the given interval
df_dict = ComputeYears(CDM,year_list)

# Display shape of data after filtering
for i in df_dict: 
    for s_type in df_dict[i]:
        print("Year: {}, survival category: {}, size: {}".format(i,s_type,df_dict[i][s_type].shape[0]))
        
YEAR = 1
STYPE = "OS"

y_features = list(['status','OS_days'])

group_dict = { 
                "group 1": {
                             "stage": ['stage 4'],\
                             'wave': {
                                         "layer 1": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2','Stage'],\
                                         "layer 2": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2',\
                                                     'T (no subgroup)', 'N (no subgroup)'],\
                                         "layer 3": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2', 'T', 'N'],\
                                         "layer 4": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2', 'size_precise', 'nodespos']
                                     }
                           },\
                "group 2": {
                             'stage': ['dcis/lcis non-invasive'],\
                             'wave': {
                                         "layer 1": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2','Size'],\
                                         "layer 2": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2','size_precise']
                                     }
                           },\
                "group 3": {
                             "stage": ['stage 4','dcis/lcis non-invasive'],\
                             'wave': {
                                         "layer 1": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2','Stage'],\
                                         "layer 2": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2',\
                                                     'T (no subgroup)', 'N (no subgroup)', 'M (no subgroup)'],\
                                         "layer 3": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2', 'T', 'N', 'M'],\
                                         "layer 4": ['Age_@_Dx', 'diff', 'ER', 'PR','Her2', 'size_precise',\
                                                     'nodespos','M']
                                     }
                           },
                }
         
model_data_dict = layeredData(df_dict, group_dict,y_features,YEAR, STYPE)

In [ ]:
for group in model_data_dict:
    for layer in model_data_dict[group]:
        print("{}_{}_{}".format(group,layer,model_data_dict[group][layer]['X'].shape))
        model_data_dict[group][layer]['X'].to_pickle("C:\\SMU_v2\\model_dict\\{}_{}.pkl".format(group,layer))

In [ ]:
tmp = CDM[["Gender","dx_date",'diff','ProgStage_AJCC8','ER','PR','Her2',\
'size_precise','nodespos','cause_of_death',\
'Date_for_DFS','Date_for_OS','Date_for_CSS',\
'Age_@_Dx','death_age','status','T (no subgroup)', 'N (no subgroup)', 'M (no subgroup)',\
'T','N','M','Stage']]

tmp.to_csv("C:\\SMU_v2\\clinical_full_data.csv",index=False)

# Bills

In [ ]:
MONTH = 30
YEAR = 365
current_year = 2020
INTEREST = 0.03

#calculate memory usage
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

def get_patient_bills(patient_id,bills_clean):
    """
    input: patient id (str), all bills (dataframe)
    output: A dataframe containing all bills of given patient
    """
    subset = bills_clean[bills_clean["Patient.ID"] == patient_id]
    return subset

    
def get_cost_timeperiod(date, patient_bills):
    """
    input: date(pd.Timestamp) and dataframe of patient's bills (1 patient only), last bill of patient
    output: from date, calculate the sum of bills [6 months before, 6 months after, and yearly until 10 years later])
    """
    try:
        ##calculate NPV of prices
        prices = patient_bills["Gross..exclude.GST."]*patient_bills["Service.Date.From.Date"].map(lambda x: (1+INTEREST)**(current_year-x.year))
        
        ##calculate different groupings of dates
        difference = patient_bills['Service.Date.From.Date'] - date
        condition1 = difference.astype("timedelta64[D]") > (-6*MONTH)
        condition2 = difference.astype("timedelta64[D]") >= 0 
        condition3 = difference.astype("timedelta64[D]") >= (6*MONTH) 
        condition4 = difference.astype("timedelta64[D]") >= (1*YEAR) 
        condition5 = difference.astype("timedelta64[D]") >= (2*YEAR) 
        condition6 = difference.astype("timedelta64[D]") >= (3*YEAR) 
        condition7 = difference.astype("timedelta64[D]") >= (4*YEAR) 
        condition8 = difference.astype("timedelta64[D]") >= (5*YEAR) 
        condition9 = difference.astype("timedelta64[D]") >= (6*YEAR) 
        condition10 = difference.astype("timedelta64[D]") >= (7*YEAR) 
        condition11 = difference.astype("timedelta64[D]") >= (8*YEAR) 
        condition12 = difference.astype("timedelta64[D]") >= (9*YEAR) 
        condition13 = difference.astype("timedelta64[D]") >= (10*YEAR) 
        before_6m = prices[condition1 & (condition2 == False)].sum()
        after_6m = prices[condition2 & (condition3 == False)].sum()
        after_1y = prices[condition3 & (condition4 == False)].sum()
        after_2y = prices[condition4 & (condition5 == False)].sum()
        after_3y = prices[condition5 & (condition6 == False)].sum()
        after_4y = prices[condition6 & (condition7 == False)].sum()
        after_5y = prices[condition7 & (condition8 == False)].sum()
        after_6y = prices[condition8 & (condition9 == False)].sum()
        after_7y = prices[condition9 & (condition10 == False)].sum()
        after_8y = prices[condition10 & (condition11 == False)].sum()
        after_9y = prices[condition11 & (condition12 == False)].sum()
        after_10y = prices[condition12 & (condition13 == False)].sum()
        
        results = [before_6m, after_6m, after_1y, after_2y, after_3y, after_4y,
               after_5y, after_6y, after_7y,after_8y, after_9y, after_10y]
        
        ##handle bills that definately cannot appear due to lack of data to differentiate it from no bills
        data_limit = patient_bills["Service.Date.From.Date"].max()
        limit = data_limit - date
        limit_int = abs(int(limit/np.timedelta64(1, 'Y')//1)) #abs not required but its there just in case
        result_limit = results[:limit_int+3]
        
        while len(result_limit) < 12:
            result_limit.append(np.NaN)
        
        return result_limit
    except:
        return [np.NaN for i in range(12)]

In [ ]:
FileToCheck = 'C:\\SMU_v2\\bills_output.pkl'

##read in the service code mappings
mappings = pd.read_excel("C:\\SMU_v2\\service code mapping_smu.xlsx")

if os.path.exists(FileToCheck):
    bills_clean = pd.read_pickle(FileToCheck)
else:
    bills_clean = pd.DataFrame()
    
    # primary set up
    xlApp = win32com.client.Dispatch("Excel.Application")
    xlApp.Interactive = False
    xlApp.Visible = False

    path = str("C:\\SMU_v2\\")
    files = os.listdir(path)

    # Put files into dataframe dict
    df_list = {}

    #require user input for password
    pwd = getpass.getpass('Enter file password: ')

    # Pick out 'xlsx' files:
    files_xls = ['Bills Data_10-12k (MASKED)v2.xlsx', 'Bills Data_12-14k (MASKED)v2.xlsx', 'Bills Data_14-16k (MASKED)v2.xlsx', 'Bills Data_16-18k (MASKED)v2.xlsx', 'Bills Data_18-20k (MASKED)v2.xlsx', 'Bills Data_1st 2k (MASKED)v2.xlsx', 'Bills Data_2-4k (MASKED)v2.xlsx', 'Bills Data_20-22k (MASKED)v2.xlsx', 'Bills Data_22-24k (MASKED)v2.xlsx', 'Bills Data_24-26k (MASKED)v2.xlsx', 'Bills Data_4-6k (MASKED)v2.xlsx', 'Bills Data_6-8k (MASKED)v2.xlsx', 'Bills Data_8-10k (MASKED)v2.xlsx', 'Bills Data_last 1k (MASKED)v2.xlsx']

    for f in files_xls:
        
        #reading of data
        first = True
        counter = 1
        xlwb = xlApp.Workbooks.Open(path+f, False, True, None, pwd)
        xlws = xlwb.Worksheets(1) 
#         last_row = 500
        last_row = xlws.UsedRange.Rows.Count
        last_col = xlws.UsedRange.Columns.Count
        n=50000
        while counter < last_row:
            print("Processing {}".format(counter))
            prev_counter = counter
            counter = min(counter+n,last_row)
            content = xlws.Range(xlws.Cells(prev_counter, 1), xlws.Cells(counter, last_col)).Value
            if first:
                print(last_row)
                first = False
                col_headers = content[0]
                data = list(content[1:])
            else:
                data = list(content[0:])
            for x in range(0,len(data)):
                data[x] = list(data[x])
                for y in range(0,len(data[x])):
                    if isinstance(data[x][y], pywintypes.TimeType):
                        temp = str(data[x][y]).rstrip("+00:00").strip()
                        data[x][y] = datetime.datetime.strptime(temp, "%Y-%m-%d")

            bills = pd.DataFrame(data, columns=col_headers)
            #cleaning of data
            ##remove all bills with gross cost of NA
            bills['Gross..exclude.GST.'].replace("NA",np.nan,inplace=True)
            bills = bills.dropna(subset=['Gross..exclude.GST.'])

            ##removal of several unused columns
            unused = ["Gender","Date.of.Birth","Net..exclude.GST.","Net.Payable","Service.Cost",\
                      "Billed.Qty","Service.Entered.Price","Doctor.Surcharge..SVC.","Total.Cost",\
                      "TOSP","Billing.Category.Description","Billing.Status.Description","Billing.Date",\
                      "Admit.Sub.Specialty.Description","Admit.Specialty.Description",\
                      "Admit.Accommodation.Category.Description","Diagnosis.Description..ICD10.",\
                      "Diagnosis.Description"]
            
            bills = bills.drop(unused,axis = 1)

            ##can consider removing these columns 
#             unused = ["Service.Category.1.Code", "Service.Category.1.Description","Service.Category.2.Code","Service.Category.2.Description",
#                      "Service.Summary.Code"]
#             bills = bills.drop(unused,axis = 1)

            ##replace all expected unknown with np.nan
            bills = bills.replace("Expected Unknown",np.nan)
            bills = bills.replace("EXPUNKNOWN",np.nan)

            ##add the processed bills to a clean df
            
            bills_clean = bills_clean.append(bills.reset_index())
            
        name = xlApp.ActiveWorkbook.Name
        #reset variables
        xlws = None
        xlwb.Close(False)
        xlwb = None
        
        ##type casting to save space
        bills_clean.loc[:,"Gross..exclude.GST."] = bills_clean["Gross..exclude.GST."].astype("uint64")
        bills_clean.loc[:,"Service.Qty"] = bills_clean["Service.Qty"].astype("uint16")
        bills_clean.loc[:,"Institution.Code"] = bills_clean["Institution.Code"].astype('category')
        bills_clean.loc[:,"Service.Summary..Description"] = bills_clean["Service.Summary..Description"].astype('category')
        bills_clean.loc[:,"Service.Code"] = bills_clean["Service.Code"].astype('category')
        bills_clean.loc[:,"Service.Short.Text"] = bills_clean["Service.Short.Text"].astype('category')
        bills_clean.loc[:,"Service.Department.Description"] = bills_clean["Service.Department.Description"].astype('category')
        bills_clean.loc[:,"Diagnosis.Code"] = bills_clean["Diagnosis.Code"].astype('category')
        bills_clean.loc[:,"Diagnosis.Code..ICD10."] = bills_clean["Diagnosis.Code..ICD10."].astype('category')

    #remove buffer and reset system settings
    xlApp.Interactive = True
    xlApp.Visible = True
    xlApp.Quit()
    xlApp = None

    bills_clean = bills_clean.drop(["index"],axis = 1)
    
    print("Done")
    
    ##save the df
    outToPickle(bills_clean,FileToCheck)

In [ ]:
FileToCheck = "C:\\SMU_v2\\price_master.pkl"

import pickle

if os.path.exists(FileToCheck):
    bills_master = pd.read_pickle(FileToCheck)
else:
    prices = []
    counter = 1
    for i,j in CDM.iterrows():
        counter+=1
        prices.append(
            get_cost_timeperiod(j["dx_date"],
                     get_patient_bills(j["NRIC"],bills_clean)))
        if counter%10 == 0:
            print(counter)
        if counter%1000 == 0:
            with open("price_processing", 'wb') as fp:
                pickle.dump(prices, fp)

    bills_master = pd.DataFrame(prices,columns=["before_6m", "after_6m", "after_1y", "after_2y", "after_3y", "after_4y",
               "after_5y", "after_6y", "after_7y","after_8y", "after_9y", "after_10y"])
    bills_master.to_pickle(FileToCheck)

### Bridge between Bills and Clinical

In [ ]:
CDM = CDM.reset_index(drop=True)
bills_master = bills_master.reset_index(drop=True)
print(CDM.shape)
print(bills_master.shape)
combined = pd.concat([CDM,bills_master],axis=1)
FileToCheck = "C:\\SMU_v2\\price_timeperiod.pkl"


if os.path.exists(FileToCheck):
    bills_processed_time = pd.read_pickle(FileToCheck)
else:
    
    bills_processed_time = combined.dropna(axis=0, \
                    subset=['Date_for_DFS','Date_for_OS','Date_for_CSS','dx_date',\
                            'Age_@_Dx','size_precise', 'nodespos'],
                    )
    bills_processed_time.to_pickle(FileToCheck)